In [1]:
pip uninstall tensorflow

Uninstalling tensorflow-2.2.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-2.2.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.2.0


In [2]:
pip install tensorflow==1.13.2

     |████████████████████████████████| 92.6MB 70kB/s 
     |████████████████████████████████| 3.2MB 47.7MB/s 
     |████████████████████████████████| 368kB 46.7MB/s 
  Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0


In [3]:
import tensorflow as tf
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.contrib import learn

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [0]:
# Input: Product description
proudct_description = "Should I buy tiago?"

In [0]:
# Absolute path to pre-trained model
model_path = '/content/model_triplet'

In [0]:

def model_output(feed_data):
    checkpoint_file = tf.train.latest_checkpoint(model_path)
    graph = tf.Graph()
    with graph.as_default():
        sess = tf.Session()
        with sess.as_default():
            # Load the saved meta graph and restore variables
            saver = tf.train.import_meta_graph("{}.meta".format(checkpoint_file))
            saver.restore(sess, checkpoint_file)

            # Get the placeholders from the graph by name
            anchor_input = graph.get_operation_by_name("left_input").outputs[0]

            # Tensors we want to evaluate
            predictions = graph.get_operation_by_name("output/output_embedding").outputs[0]

            # Collect the predictions here
            all_predictions = []

            batch_predictions = sess.run(predictions, {anchor_input: feed_data})
    return batch_predictions

In [0]:
vocab_processor = learn.preprocessing.VocabularyProcessor.restore(os.path.join(model_path,'vocab'))
query = list(proudct_description.lower())
input_queries = np.asarray(list(vocab_processor.fit_transform(query)))

In [11]:
# Embeddings output. If you want to use this embedding to calculate Cosine Similariy use normalized_search_vec otherwise use search_vec
search_vector = model_output([input_queries[0]])
normalized_search_vec = search_vector/np.linalg.norm(search_vector)
# normalized item embedding
sentence_1 = normalized_search_vec

INFO:tensorflow:Restoring parameters from /content/model_triplet/model.ckpt


In [12]:
#query
description = "I'm a triple Capricorn"
queryy = list(description.lower())
input_queriess = np.asarray(list(vocab_processor.fit_transform(queryy)))
search_vectorr = model_output([input_queriess[0]])
sentence_2 = search_vectorr/np.linalg.norm(search_vectorr)

INFO:tensorflow:Restoring parameters from /content/model_triplet/model.ckpt


In [13]:
sim =  cosine_similarity(sentence_1,sentence_2)
sim[0][0]

0.2429741

In [0]:
import pandas as pd

In [0]:
dataset = pd.read_csv('SA.csv')

In [0]:
def removing_bad_characters(test_string):
   bad_chars = [';', ':', '!', "*",'?','"',"'",'>','<','•','o','≤','â','€','¢'] 
   # remove bad_chars  
   for i in bad_chars : 
      test_string = test_string.replace(i, '') 
   return test_string

In [0]:
similar_pairs = pd.DataFrame()
similar_pairs['Possible Questions'] = dataset['Question'].str.lower()
similar_pairs['Possible Questions'] = removing_bad_characters(similar_pairs['Possible Questions'])
similar_pairs['Answer'] = dataset['Answer'].str.lower()
similar_pairs['Answer'] = removing_bad_characters(similar_pairs['Answer'])

In [0]:
def removing_bad_characters(test_string):
   bad_chars = ['\n', '\t'] 
   # remove bad_chars  
   for i in bad_chars : 
      test_string = test_string.replace(i, '') 
   return test_string

In [37]:
dataset = similar_pairs
import nltk
from nltk.corpus import reuters
nltk.download("reuters")
nltk.download("punkt")
max_samples = 256
Questions = []
True_Atom = []
false = []
for i in range(dataset.shape[0]):
  Questions.append(removing_bad_characters(dataset['Possible Questions'][i]))
  True_Atom.append(removing_bad_characters(dataset['Answer'][i]))
S, X = [], []
queryy = list(Questions[2].lower())
input_queriess = np.asarray(list(vocab_processor.fit_transform(queryy)))
search_vectorr = model_output([input_queriess[0]])
sentence_2 = search_vectorr/np.linalg.norm(search_vectorr)
X.append(sentence_2)
S.append(Questions[2])
for i in range(10):
  queryy = list(True_Atom[i].lower())
  input_queriess = np.asarray(list(vocab_processor.fit_transform(queryy)))
  search_vectorr = model_output([input_queriess[0]])
  sentence_2 = search_vectorr/np.linalg.norm(search_vectorr)
  X.append(sentence_2)
  S.append(True_Atom[i])
X = np.vstack(X) 
X.shape

[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
INFO:tensorflow:Restoring parameters from /content/model_triplet/model.ckpt
INFO:tensorflow:Restoring parameters from /content/model_triplet/model.ckpt
INFO:tensorflow:Restoring parameters from /content/model_triplet/model.ckpt
INFO:tensorflow:Restoring parameters from /content/model_triplet/model.ckpt
INFO:tensorflow:Restoring parameters from /content/model_triplet/model.ckpt
INFO:tensorflow:Restoring parameters from /content/model_triplet/model.ckpt
INFO:tensorflow:Restoring parameters from /content/model_triplet/model.ckpt
INFO:tensorflow:Restoring parameters from /content/model_triplet/model.ckpt
INFO:tensorflow:Restoring parameters from /content/model_triplet/model.ckpt
INFO:tensorflow:Restoring parameters from /content/model_triplet/model.ckpt
INFO:tensorflo

(11, 64)

In [0]:
Y=['Q',0,0,0,0,0,0,0,0,0,1]

In [0]:
with open("metadata1.tsv", "w") as fo:
  fo.write("Label\tSentence\n")
  for y, s in zip(Y, S):
    fo.write("{}\t{}\n".format(y, s))

In [0]:
with open("embeddings1.tsv", "w") as fo:
  for x in X.astype('float16'):
    line = "\t".join([str(v) for v in x])
    fo.write(line + "\n")